# Rock Paper Scissors Regret Matching

Worked example from: http://modelai.gettysburg.edu/2013/cfr/cfr.pdf

The high level algorithm is as follows:

- For each player, init all cumulative regrets to 0
- For some number of iterations:
  - Compute a regret-matching strategy profile (if all nonpositive, uniformly random)
  - Add strategy profile to strategy profile sum 
  - Select each player action profile according to strategy profile
  - Compute player regrets
  - Add player regrets to player cumulative regrets
- Return average strategy profile 

## Definitions

First some definitions: 

- Arbitrarily assign the values 0, 1, 2 to Rock, Paper, Scissors. 
- Create a RNG
- Allocate arrays to hold:
  - Accumulated action regrets
  - A strategy generated through regret-matching
  - Sum of all such strategies generated
- Number of iterations we want to run this algo for

In [1]:
from random import random

ROCK, PAPER, SCISSORS, NUM_ACTIONS = 0, 1, 2, 3

regretSum = [0.0] * NUM_ACTIONS 
strategy = [0.0] * NUM_ACTIONS
strategySum = [0.0] * NUM_ACTIONS
oppStrategy = [0.4, 0.3, 0.3]

## Get Current Mixed Strategy Though Regret-Matching

We want to select actions in proportion to positive regrets of not having chosen them.

- Copy all positive regrets and sum them 
- Second pass through strategy entries
  - If there is at least one action with positive regret, normalize regrets 

In [2]:
# REQUIRES: True
# ENSURES:  Get's a strategy lol?

def getStrategy():
    normalizingSum = 0
    for a in range(NUM_ACTIONS):
        strategy[a] = regretSum[a] if regretSum[a] > 0 else 0
        normalizingSum += strategy[a]
    for a in range(NUM_ACTIONS):
        if (normalizingSum > 0):
            strategy[a] /= normalizingSum
        else:
            strategy[a] = 1.0 / NUM_ACTIONS # if all nonpositive, make uniform
        strategySum[a] += strategy[a]
    return strategy 

## Get Random Action According to Mixed-Strategy Distribution

Now that we have a mixed strategy, we want to randomly pick an action. 

- Randomly generate a probability from (0,1]
- Check all actions and see if it's less than random probability 
    - If so, we select that action
    - Otherwise return the last action

In [3]:
# REQUIRES: strategy is a float list
# ENSURES:  a is a randomly picked action according 
#           to our mixed strategy distribution

def getAction(strategy): 
    # COME BACK TO THIS ?????
    r = random() # we do 1 - since python random gives [0,1)
    a = 0
    cumulativeProb = 0.0
    while(a < NUM_ACTIONS - 1):
        cumulativeProb += strategy[a]
        if(r < cumulativeProb):
            break
        a += 1
    return a

## Training

We have three parts to our traning algorithm

1. Get Regret-matched mixed-strategy actions
2. Compute action utilities 
3. Accumulate action regrets

In [4]:
# REQUIRES: iterations the int number of times u wanna run this
# ENSURES:  void, updates the things accordingly ??

def train(iterations):
    
    actionUtility = [0.0] * NUM_ACTIONS 

    for i in range(iterations):
        
        # 1) Get Regret-matched mixed-strategy actions
        strategy = getStrategy()
        myAction = getAction(strategy)
        otherAction = getAction(oppStrategy)

        # 2) Compute action utilities
        # This is some big brain stuff with modding. 
        # May hardcode this later for elements
        actionUtility[otherAction] = 0
        actionUtility[0 if otherAction == NUM_ACTIONS - 1 else otherAction + 1] = 1
        actionUtility[NUM_ACTIONS - 1 if otherAction == 0 else otherAction - 1] = -1

        # 3) Accumulate action regrets
        for a in range(NUM_ACTIONS):
            regretSum[a] += actionUtility[a] - actionUtility[myAction]
            
        if(i % 10000 == 0):
            print("Iteration " + str(i))
            print(strategy)

## Get Average Mixed Strategy Across All Training Iterations

In [5]:
# REQUIRES: strategy is trained
# ENSURES:  returns a float list representing average strategy

def getAverageStrategy():
    
    avgStrategy = [0.0] * NUM_ACTIONS
    normalizingSum = 0
    
    for a in range(NUM_ACTIONS):
        normalizingSum += strategySum[a]
    for a in range(NUM_ACTIONS):
        if (normalizingSum > 0):
            avgStrategy[a] = strategySum[a] / normalizingSum
        else:
            avgStrategy[a] = 1.0 / NUM_ACTIONS
            
    return avgStrategy

## Main Method Initializing Computation

In [6]:
def main():
    train(100000)
    print("Iterations Averaged Out")
    print(getAverageStrategy())

In [7]:
main()

Iteration 0
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Iteration 10000
[0.0, 1.0, 0.0]
Iteration 20000
[0.0, 1.0, 0.0]
Iteration 30000
[0.0, 1.0, 0.0]
Iteration 40000
[0.0, 1.0, 0.0]
Iteration 50000
[0.0, 1.0, 0.0]
Iteration 60000
[0.0, 1.0, 0.0]
Iteration 70000
[0.0, 1.0, 0.0]
Iteration 80000
[0.0, 1.0, 0.0]
Iteration 90000
[0.0, 1.0, 0.0]
Iterations Averaged Out
[0.0029810198412698412, 0.9950299801587302, 0.001989]
